In [15]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv('/home/shua/Desktop/msc-ml-datamining/Machine_Learning/project/v2/data/prepared_data.csv')

In [17]:
df = df.dropna(subset = ['prev_sale_price'])

In [18]:
from sklearn.model_selection import train_test_split

#drop columns that are address related
train_df = df.loc[:, ~df.columns.str.contains('^address')]
train_df = train_df.drop(['street', 'city', 'state', 'zip'], axis = 1)
#drop years (if calculated the age)
train_df = train_df.drop(['prev_sale_price', 'sale.lastSale.saleDate', 'sale.priorSale.saleDate', 'building.yearBuilt', 'sold_month'], axis = 1)
#split target
target = train_df.loc[:, 'sale.lastSale.price']
train_df = train_df.drop(['sale.lastSale.price'], axis = 1)

x_train, x_test, y_train, y_test = train_test_split(train_df, target, test_size=0.2, random_state=42)


#### Fill Missing Values: Fill missing values based off of column-wise aggregations on the training set

Generally, we will any remaining empty values based on column wise aggregations (such as mode or mean). Best practice is to just do this for the training sets, and use the modes and mean (or other metrics) of the training set for the test set (utilising the fit_transform methods of sklearn).

In [19]:
# from sklearn.base import BaseEstimator, TransformerMixin

# #create a Datafill class to fill in the "small_missing" missing values with the mode from the training set, even on prediction
# class DataFill(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         pass
#     def fit(self, X, y = None):
#         self.columns = X.columns
#         self.modes_ = {}
#         for col in self.columns:
#             self.modes_[col] = X[col].mode()[0]
#         return self
#     def transform(self, X, y = None):
#         X = X.copy()
#         for col in self.columns:
#             X[col].fillna(self.modes_[col], inplace = True)
#         return X

# fill = DataFill()
# fill.fit(df[small_missing])
# df[small_missing] = fill.transform(df[small_missing])


Using Min-max instead of standard scaling, because of the many one-hot encodings (need a source)

In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [21]:
#get the log of the target
# y_train = np.log(y_train)
# y_test = np.log(y_test)

In [22]:
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imputer = IterativeImputer(max_iter=10, random_state=0)
# imputer.fit(x_train)
# x_train_impute = pd.DataFrame(imputer.transform(x_train), columns=train_df.columns)


#### Imputation: Use KNN to fill in the rest of the missing values

In [23]:
#implement knn imputer
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=4)
imputer.fit(x_train)
x_train = pd.DataFrame(imputer.transform(x_train), columns=train_df.columns)
x_test = pd.DataFrame(imputer.transform(x_test), columns=train_df.columns)

In [24]:
#print out the columns that have null values
x_train.isnull().sum()[x_train.isnull().sum() > 0]

Series([], dtype: int64)

TODO: Need to group unique houses into same train and test splits. Otherwise, risk overfitting